<a href="https://colab.research.google.com/github/vstiwari/Feature-Selection/blob/main/IBPSO_Algo_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np

def s_rel_corr(X,y,target='default payment next month'):
  import scipy.stats as ss

  D=X.shape[1]

  if isinstance(X, pd.DataFrame):
    MI=np.array(np.abs(pd.concat([y,X],axis=1).corr()[target][1:]))
  else:
    MI=np.array(np.abs(pd.DataFrame(np.concatenate((y.reshape([len(y),1]),X),axis=1)).corr()[target][1:]))

  # mutual_info_classif(X,y)


  rankk=int(D/np.log(D))


  rankdata=ss.rankdata(MI,method='ordinal')
  # print(MI,rankdata,rankk)
  min2=MI[rankdata==rankk]
  min1=0.05*max(MI)
  minim=min([min1,min2])
  minim=0
  var_list=[m for m in range(len(MI)) if MI[m]>minim]
  if isinstance(X, pd.DataFrame):
    var_list=[X.columns[m] for m in range(len(MI)) if MI[m]>minim]



  return var_list,[MI[m] for m in range(len(MI)) if MI[m]>minim],MI


def corr_one_column(x,X,col_num):
  if isinstance(X, pd.DataFrame):
    r=np.corrcoef(x,X.loc[:,col_num])[0,1]
  else:
    r=np.corrcoef(x,X[:,col_num])[0,1]
  return r

def clustering_corr(X1,y,rel_list,MI_step1):


  if isinstance(X1, pd.DataFrame):
    # print("it works")

    X=X1.to_numpy()


  clustering_table=pd.DataFrame(columns=['cols','MI','MI_anchor_diff','non_corr','MI_fifj','MI_fancC','clustered1','clustered2','final_clustered','cluster_no'])
  MI_c=pd.DataFrame({'cols':rel_list,'MI':MI_step1})
  iter=0
  while(MI_c.shape[0]>0):
    MI_c_iter=MI_c.copy()
    feature_anchor_id=MI_c_iter['MI'].idxmax()

    MI_c_iter['MI_anchor_diff']=MI_c_iter.loc[feature_anchor_id,'MI']-MI_c_iter.loc[:,'MI']

    d_dash=MI_c_iter.shape[0]
    p1=MI_c_iter['MI_anchor_diff'].max()*np.log(d_dash)*3/d_dash

    MI_c_iter['non_corr']=np.where(MI_c_iter['MI_anchor_diff']>p1,1,0)

    if isinstance(X1, pd.DataFrame):
      # MI_c_iter['MI_fifj']=mutual_info_regression(X1.loc[:,MI_c_iter['cols']],X1.loc[:,MI_c_iter.loc[feature_anchor_id,'cols']])
      # MI_c_iter['MI_fifj']=X1.loc[:,MI_c_iter['cols']].corr()[MI_c_iter.loc[feature_anchor_id,'cols']]
      MI_c_iter['MI_fifj']=list(np.abs(X1.loc[:,MI_c_iter['cols']].apply(corr_one_column,X=X1,col_num=MI_c_iter.loc[feature_anchor_id,'cols'],axis=0)))


    else:
      # MI_c_iter['MI_fifj']=mutual_info_regression(X[:,MI_c_iter['cols']],X[:,MI_c_iter.loc[feature_anchor_id,'cols']])
      # MI_c_iter['MI_fifj']=X1[:,MI_c_iter['cols']].corr()[MI_c_iter.loc[feature_anchor_id,'cols']]
      # MI_c_iter['MI_fifj']=X1.apply(corr_one_column,X=X1,col_num=MI_c_iter.loc[feature_anchor_id,'cols'],axis=0)
      MI_c_iter['MI_fifj']=list(np.abs(pd.DataFrame(X1[:,MI_c_iter['cols']]).apply(corr_one_column,X=X1,col_num=MI_c_iter.loc[feature_anchor_id,'cols'],axis=0)))

    MI_c_iter['MI_fancC']=MI_c_iter.loc[feature_anchor_id,'MI']


    MI_c_iter['clustered1']=np.where((MI_c_iter['MI_fifj']>MI_c_iter['MI']),1,0)
    MI_c_iter['clustered2']=np.where((MI_c_iter['MI_fifj']>MI_c_iter.loc[feature_anchor_id,'MI']),1,0)
    MI_c_iter['final_clustered']=np.where(np.logical_or((MI_c_iter['clustered1']==1),(MI_c_iter['clustered2']==1)),1,0)

    MI_c_iter['cluster_no']=iter

    clustering_table=pd.concat([clustering_table,MI_c_iter[(MI_c_iter['non_corr']==0) & (MI_c_iter['final_clustered']==1)]],axis=0)


    MI_c.drop(MI_c_iter.loc[(MI_c_iter['non_corr']==0) & (MI_c_iter['final_clustered']==1),:].index,axis=0,inplace=True)


    iter=iter+1

  clustering_table['code']=clustering_table.groupby('cluster_no').cumcount()+1
  clustering_table['prob_select']=clustering_table['MI_fancC']/clustering_table['MI_fancC'].max()
  return clustering_table

In [5]:
def obj_fun(table,code_list):
  from sklearn.linear_model import LogisticRegression
  from sklearn.metrics import accuracy_score
  clust_num=len(table['cluster_no'].unique())
  l=[]
  for i in range(clust_num):
    if code_list[i]!=0:
      l.append(list(table.loc[(table['code']==code_list[i]) & (table['cluster_no']==i),'cols'])[0])

  X1=X.loc[:,l]

  clf = LogisticRegression(random_state=0).fit(X1, y)
  return accuracy_score(y,clf.predict(X1))

def initializer(table):
  clust_num=len(table['cluster_no'].unique())

  l=[]
  for i in range(clust_num):
    table1=table[table['cluster_no']==i]
    if np.random.rand()<list(table1['prob_select'])[0]:
      l.append(int(np.random.uniform(0,table[table['cluster_no']==i]['code'].max()+1,1)))
    else:
      l.append(0)

  #handling no feature selection case
  sel_check=np.unique(np.array(l))
  if (len(sel_check)==1) & (sel_check[0]==0):
    l[table.iloc[table['MI'].idxmax(),:]['cluster_no']]=table.iloc[table['MI'].idxmax(),:]['code']
  return l

In [4]:
def IBPSO(X,table,particles=10,iters=5):
  num_each_clust_limit=table.groupby(['cluster_no'])['cluster_no'].count()

  clust_num=len(table['cluster_no'].unique())
  initial_values=pd.DataFrame(columns=['part'+str(i) for i in range(particles)])
  for i in range(particles):
    initial_values[initial_values.columns[i]]=initializer(table)
    # print(initial_values)
    # initial_values=pd.concat([initial_values,pd.DataFrame(initializer(table),columns=['part'+str(i)])],axis=1)

  p_best=np.zeros([particles])
  p_best_state=initial_values

  g_best=0
  g_best_state=np.empty([clust_num])

  current_value=np.empty([particles])
  current_state=initial_values

  # print(current_state)
  for i in range(particles):
    current_value[i]=obj_fun(table,initial_values.iloc[:,i])
    current_state.iloc[:,i]=initial_values.iloc[:,i]

    if p_best[i]<current_value[i]:
      p_best[i]=current_value[i]
      p_best_state.iloc[:,i]=current_state.iloc[:,i]

    if (np.round(p_best[i],2)==np.round(current_value[i])) & np.sum(current_state.iloc[:,i]!=0)<np.sum(p_best_state.iloc[:,i]!=0):
      p_best[i]=current_value[i]
      p_best_state.iloc[:,i]=current_state.iloc[:,i]


  if np.max(p_best)>g_best:
    g_best=np.max(p_best)
    g_best_state=p_best_state.iloc[:,np.argmax(p_best)]

  if (np.round(np.max(p_best))==np.round(g_best)) & (np.sum(p_best_state.iloc[:,i]!=0)<np.sum(g_best_state!=0)):
    g_best=np.max(p_best)
    g_best_state=p_best_state.iloc[:,np.argmax(p_best)]


  import math

  for i in range(particles):

    p_best_state1=p_best_state.copy()

    # val1=((p_best_state1.iloc[:,i]+g_best_state)/2).apply(math.ceil)+(np.abs(np.random.normal())*(np.abs(p_best_state1.iloc[:,i]-g_best_state))).apply(math.ceil)
    val1=((p_best_state1.iloc[:,i]+g_best_state)/2).apply(math.ceil)+((np.random.normal())*(np.abs(p_best_state1.iloc[:,i]-g_best_state))).apply(math.ceil)
    val2=p_best_state1.iloc[:,i]
    current_state.iloc[:,i]=np.where(np.random.rand()>0.5,val1,val2)


    if np.all(p_best_state.iloc[:,i]==g_best_state):
      current_state.iloc[:,i]=np.where(np.random.rand()<0.1/clust_num,initializer(table),current_state.iloc[:,i])

    # print((np.all(current_state.iloc[:,i]<=num_each_clust_limit)==False) | (np.all(current_state.iloc[:,i]>=0)==False))
    if np.all(current_state.iloc[:,i]<=num_each_clust_limit)==False:
      current_state.iloc[:,i]=p_best_state1.iloc[:,i]

    if np.all(current_state.iloc[:,i]>=0)==False:
      current_state.iloc[:,i]=p_best_state1.iloc[:,i]

    if np.all(current_state.iloc[:,i]==0):
      current_state.iloc[:,i]=p_best_state1.iloc[:,i]
    # print('new val',list(current_state[current_state.columns[i]].astype(int)))

    current_value[i]=obj_fun(table,current_state.iloc[:,i])

  # print(current_state)

  for j in range(iters):
    for i in range(particles):

      if p_best[i]<current_value[i]:
        p_best[i]=current_value[i]
        p_best_state.iloc[:,i]=current_state.iloc[:,i]

      if (np.round(p_best[i],2)==np.round(current_value[i],2)) & (np.sum(current_state.iloc[:,i]!=0)<np.sum(p_best_state.iloc[:,i]!=0)):
        p_best[i]=current_value[i]
        p_best_state.iloc[:,i]=current_state.iloc[:,i]


    if np.max(p_best)>g_best:
      g_best=np.max(p_best)
      g_best_state=p_best_state.iloc[:,np.argmax(p_best)]

    if (np.round(np.max(p_best))==np.round(g_best)) & np.sum(p_best_state.iloc[:,i]!=0)<np.sum(g_best_state!=0):
      g_best=np.max(p_best)
      g_best_state=p_best_state.iloc[:,np.argmax(p_best)]


    import math
    for i in range(particles):

      p_best_state1=p_best_state.copy()

      # val1=((p_best_state1.iloc[:,i]+g_best_state)/2).apply(math.ceil)+(np.abs(np.random.normal())*(np.abs(p_best_state1.iloc[:,i]-g_best_state))).apply(math.ceil)
      val1=((p_best_state1.iloc[:,i]+g_best_state)/2).apply(math.ceil)+((np.random.normal())*(np.abs(p_best_state1.iloc[:,i]-g_best_state))).apply(math.ceil)

      val2=p_best_state1.iloc[:,i]
      current_state.iloc[:,i]=np.where(np.random.rand()>0.5,val1,val2)


      if np.all(p_best_state.iloc[:,i]==g_best_state):
        # current_state.iloc[:,i]=np.where(np.random.rand()<0.1/clust_num,initializer(table),current_state.iloc[:,i])
        current_state.iloc[:,i]=np.where(np.random.rand()<0.1,initializer(table),current_state.iloc[:,i])

      # print(np.all(current_state.iloc[:,i]<=num_each_clust_limit)==False)
      if np.all(current_state.iloc[:,i]<=num_each_clust_limit)==False:
        current_state.iloc[:,i]=p_best_state1.iloc[:,i]

      if np.all(current_state.iloc[:,i]>=0)==False:
        current_state.iloc[:,i]=p_best_state1.iloc[:,i]

      if np.all(current_state.iloc[:,i]==0):
        current_state.iloc[:,i]=p_best_state1.iloc[:,i]


      current_value[i]=obj_fun(table,current_state.iloc[:,i])

    # print(p_best)
    print(g_best,np.sum(g_best_state!=0))




In [26]:
from sklearn.datasets import make_classification
data=make_classification(n_samples=100, n_features=200, n_informative=20, n_redundant=180, n_repeated=0, n_classes=2, weights=None, flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=None)
X=pd.DataFrame(data[0],columns=['a'+str(i) for i in range(data[0].shape[1])])
y=pd.DataFrame(data[1],columns=['target'])


In [27]:

rel_list,MI_step1,MI=s_rel_corr(X,y,target='target')
table=clustering_corr(X,y,rel_list,MI_step1)

In [28]:
table

,cols,MI,MI_anchor_diff,non_corr,MI_fifj,MI_fancC,clustered1,clustered2,final_clustered,cluster_no,code,prob_select
64,a64,0.543386,0.034700,0,0.647386,0.578086,1,1,1,0,1,1.000000
101,a101,0.578086,0.000000,0,1.000000,0.578086,1,1,1,0,2,1.000000
121,a121,0.565337,0.000000,0,1.000000,0.565337,1,1,1,1,1,0.977946
72,a72,0.511574,0.000000,0,1.000000,0.511574,1,1,1,2,1,0.884945
65,a65,0.492625,0.017843,0,0.505850,0.510468,1,0,1,3,1,0.883031
...,...,...,...,...,...,...,...,...,...,...,...,...
49,a49,0.002714,0.013942,0,0.008469,0.016656,1,0,1,48,2,0.028812
102,a102,0.004457,0.012199,0,0.088121,0.016656,1,1,1,48,3,0.028812
123,a123,0.003022,0.013634,0,0.193228,0.016656,1,1,1,48,4,0.028812
164,a164,0.009398,0.007258,0,0.221522,0.016656,1,1,1,48,5,0.028812


In [29]:
import warnings
warnings.filterwarnings("ignore")
IBPSO(X,table,particles=50,iters=100)

0.98 24 [0, 0, 1, 2, 0, 0, 1, 1, 2, 0, 1, 0, 0, 2, 1, 0, 0, 1, 0, 1, 4, 2, 2, 2, 4, 0, 0, 0, 0, 0, 1, 3, 0, 3, 1, 2, 3, 0, 6, 0, 0, 6, 0, 3, 0, 0, 0, 0, 0, 0]
0.98 31
0.98 36
0.98 36
0.98 36
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33
0.98 33


In [22]:
def IBPSO(X,table,particles=10,iters=5):
  import math
  num_each_clust_limit=table.groupby(['cluster_no'])['cluster_no'].count()
  rounding_factor=1/(num_each_clust_limit+1)
  clust_num=len(table['cluster_no'].unique())
  current_state=pd.DataFrame(columns=['part'+str(i) for i in range(particles)])

  current_state_raw=pd.DataFrame(columns=['part'+str(i) for i in range(particles)])

  current_value=np.empty(particles)

  for i in range(particles):
    current_state[current_state.columns[i]]=initializer(table)
    current_value[i]=obj_fun(table,current_state.iloc[:,i])

  p_best=current_value.copy()
  p_best_state=current_state.copy()



  g_best=0
  if np.max(p_best)>g_best:
    g_best=np.max(p_best)
    g_best_state=p_best_state.iloc[:,np.argmax(p_best)]

  if (np.round(np.max(p_best),2)==np.round(g_best,2)) & np.sum(p_best_state.iloc[:,i]!=0)<np.sum(g_best_state!=0):
    g_best=np.max(p_best)
    g_best_state=p_best_state.iloc[:,np.argmax(p_best)]

  print(g_best,np.sum(g_best_state!=0),list(g_best_state))

  for j in range(iters):
    for i in range(particles):
      p_best_state1=p_best_state.copy()

      val1=((p_best_state1.iloc[:,i]+g_best_state)/2).apply(math.ceil)+((np.random.normal())*(np.abs(p_best_state1.iloc[:,i]-g_best_state))).apply(math.ceil)

      val2=p_best_state1.iloc[:,i]
      current_state.iloc[:,i]=np.where(np.random.rand()>0.5,val1,val2)


      if np.all(p_best_state.iloc[:,i]==g_best_state):
        # current_state.iloc[:,i]=np.where(np.random.rand()<0.1/clust_num,initializer(table),current_state.iloc[:,i])
        current_state.iloc[:,i]=np.where(np.random.rand()<0.1,initializer(table),current_state.iloc[:,i])

      # print(np.all(current_state.iloc[:,i]<=num_each_clust_limit)==False)
      if np.all(current_state.iloc[:,i]<=num_each_clust_limit)==False:
        current_state.iloc[:,i]=p_best_state1.iloc[:,i]

      if np.all(current_state.iloc[:,i]>=0)==False:
        current_state.iloc[:,i]=p_best_state1.iloc[:,i]

      if np.all(current_state.iloc[:,i]==0):
        current_state.iloc[:,i]=p_best_state1.iloc[:,i]


      current_value[i]=obj_fun(table,current_state.iloc[:,i])

      if p_best[i]<current_value[i]:
        p_best[i]=current_value[i]
        p_best_state.iloc[:,i]=current_state.iloc[:,i]

      if (np.round(p_best[i],2)==np.round(current_value[i],2)) & (np.sum(current_state.iloc[:,i]!=0)<np.sum(p_best_state.iloc[:,i]!=0)):
        p_best[i]=current_value[i]
        p_best_state.iloc[:,i]=current_state.iloc[:,i]


    if np.max(p_best)>g_best:
      g_best=np.max(p_best)
      g_best_state=p_best_state.iloc[:,np.argmax(p_best)]

    if (np.round(np.max(p_best))==np.round(g_best)) & np.sum(p_best_state.iloc[:,i]!=0)<np.sum(g_best_state!=0):
      g_best=np.max(p_best)
      g_best_state=p_best_state.iloc[:,np.argmax(p_best)]

    print(g_best,np.sum(g_best_state!=0))


In [25]:
import warnings
warnings.filterwarnings("ignore")
IBPSO(X,table,particles=30,iters=50)

0.96 118 [1, 1, 1, 1, 1, 1, 0, 2, 0, 1, 1, 0, 1, 0, 0, 2, 1, 1, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 0, 0, 2, 1, 4, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 2, 3, 2, 0, 2, 1, 3, 1, 4, 1, 0, 1, 0, 1, 0, 0, 2, 6, 0, 1, 1, 4, 1, 1, 2, 0, 0, 2, 2, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 5, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 3, 0, 0, 0, 2, 0, 2, 0, 0, 5, 3, 0, 1, 4, 3, 0, 5, 0, 0, 2, 1, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 1, 5, 0, 2, 0, 2, 2, 3, 3, 0, 0, 0, 0, 6, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 6, 0, 0, 0, 1, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 9, 0, 8, 0, 0, 4, 6, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 8, 0, 1, 8, 4, 1, 0, 0, 9, 0, 0, 0, 11, 0, 6, 0, 0, 0, 8, 11, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 3, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 5, 0, 2, 0, 0, 0, 0, 0, 16, 2, 0, 0, 0, 0, 0, 0, 0, 0, 

In [30]:
particles=20
iters=50
num_each_clust_limit=table.groupby(['cluster_no'])['cluster_no'].count()
rounding_factor=1/(num_each_clust_limit+1)
clust_num=len(table['cluster_no'].unique())
current_state=pd.DataFrame(columns=['part'+str(i) for i in range(particles)])

current_state_raw=pd.DataFrame(columns=['part'+str(i) for i in range(particles)])

current_value=np.empty(particles)

for i in range(particles):
  current_state[current_state.columns[i]]=initializer(table)

  current_state_raw[current_state_raw.columns[i]]=np.where(current_state[current_state.columns[i]]==0,num_each_clust_limit+1,
                                                           current_state[current_state.columns[i]])
  current_state_raw[current_state_raw.columns[i]]=current_state_raw[current_state_raw.columns[i]]*rounding_factor-0.01
  current_state_raw[current_state_raw.columns[i]]=np.log(current_state_raw[current_state_raw.columns[i]]/(1-current_state_raw[current_state_raw.columns[i]]))

  current_value[i]=obj_fun(table,current_state.iloc[:,i])

p_best=current_value.copy()
p_best_state=current_state.copy()
p_best_state_raw=current_state_raw.copy()

g_best=0
if np.max(p_best)>g_best:
  g_best=np.max(p_best)
  g_best_state=p_best_state.iloc[:,np.argmax(p_best)]
  g_best_state_raw=p_best_state_raw.iloc[:,np.argmax(p_best)]

if (np.round(np.max(p_best),2)==np.round(g_best,2)) & np.sum(p_best_state.iloc[:,i]!=0)<np.sum(g_best_state!=0):
  g_best=np.max(p_best)
  g_best_state=p_best_state.iloc[:,np.argmax(p_best)]
  g_best_state_raw=p_best_state_raw.iloc[:,np.argmax(p_best)]

print(g_best,np.sum(g_best_state!=0),list(g_best_state))

velocity=pd.DataFrame(np.random.random((current_state.shape)),columns=current_state.columns)
for j in range(iters):
  for i in range(particles):

    # print('old val',list(current_state[current_state.columns[i]].astype(int)))
    velocity[velocity.columns[i]]=0.8*velocity[velocity.columns[i]]+np.random.rand()*0.1*(p_best_state_raw.iloc[:,i]-current_state_raw.iloc[:,i])
    +np.random.rand()*0.1*(g_best_state_raw-current_state_raw.iloc[:,i])

    current_state_raw[current_state_raw.columns[i]]=current_state_raw[current_state_raw.columns[i]]+10*velocity[velocity.columns[i]]

    current_state[current_state.columns[i]]=1/(1+np.exp(-current_state_raw[current_state_raw.columns[i]]))

    current_state[current_state.columns[i]]=np.ceil(current_state[current_state.columns[i]]/rounding_factor)

    current_state[current_state.columns[i]]=np.where(current_state[current_state.columns[i]]==num_each_clust_limit+1,0,
                                                     current_state[current_state.columns[i]])

    if np.all(p_best_state.iloc[:,i]==g_best_state):
      current_state.iloc[:,i]=np.where(np.random.rand()<0.1/clust_num,initializer(table),current_state.iloc[:,i])
      # current_state.iloc[:,i]=np.where(np.random.rand()<0.1,initializer(table),current_state.iloc[:,i])
      current_state_raw[current_state_raw.columns[i]]=np.where(current_state[current_state.columns[i]]==0,num_each_clust_limit+1,current_state[current_state.columns[i]])
      current_state_raw[current_state_raw.columns[i]]=current_state_raw[current_state_raw.columns[i]]*rounding_factor-0.01
      current_state_raw[current_state_raw.columns[i]]=np.log(current_state_raw[current_state_raw.columns[i]]/(1-current_state_raw[current_state_raw.columns[i]]))


    # if np.all(current_state.iloc[:,i]<=num_each_clust_limit)==False:
    #   current_state.iloc[:,i]=p_best_state[p_best_state.columns[i]]
    #   current_state_raw.iloc[:,i]=p_best_state_raw[p_best_state_raw.columns[i]]

    # if np.all(current_state.iloc[:,i]>=0)==False:
    #   current_state.iloc[:,i]=p_best_state[p_best_state.columns[i]]
    #   current_state_raw.iloc[:,i]=p_best_state_raw[p_best_state_raw.columns[i]]

    if np.all(current_state.iloc[:,i]==0):
      current_state[current_state.columns[i]]=p_best_state[p_best_state.columns[i]]
      current_state_raw.iloc[:,i]=p_best_state_raw[p_best_state_raw.columns[i]]


    current_value[i]=obj_fun(table,current_state[current_state.columns[i]])
    # print('new val',list(current_state[current_state.columns[i]].astype(int)))

    # print(p_best[i]<current_value[i])
    if p_best[i]<current_value[i]:
      # print(p_best[i],current_value[i])
      p_best[i]=current_value[i]

      p_best_state.iloc[:,i]=current_state[current_state.columns[i]]
      p_best_state_raw.iloc[:,i]=current_state_raw[current_state_raw.columns[i]]


    if (np.round(p_best[i])==np.round(current_value[i])) & (np.sum(current_state.iloc[:,i]!=0)<np.sum(p_best_state.iloc[:,i]!=0)):
      # print(p_best[i],current_value[i])
      p_best[i]=current_value[i]
      p_best_state.iloc[:,i]=current_state[current_state.columns[i]]
      p_best_state_raw.iloc[:,i]=current_state_raw[current_state_raw.columns[i]]

  if np.max(p_best)>g_best:
    g_best=np.max(p_best)
    g_best_state=p_best_state.iloc[:,np.argmax(p_best)]
    g_best_state_raw=p_best_state_raw.iloc[:,np.argmax(p_best)]

  if (np.round(np.max(p_best))==np.round(g_best)) & (np.sum(p_best_state.iloc[:,i]!=0)<np.sum(g_best_state!=0)):
    g_best=np.max(p_best)
    g_best_state=p_best_state.iloc[:,np.argmax(p_best)]
    g_best_state_raw=p_best_state_raw.iloc[:,np.argmax(p_best)]

  # print(p_best)
  print(g_best,np.sum(g_best_state!=0),list(g_best_state))
    # print(p_best_state[p_best_state.columns[i]])


0.98 20 [0, 1, 1, 2, 0, 1, 0, 1, 1, 2, 0, 0, 4, 0, 2, 2, 2, 0, 1, 0, 0, 0, 0, 5, 1, 2, 0, 0, 2, 0, 0, 0, 3, 2, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.97 18 [1, 0, 1, 1, 0, 2, 1, 1, 3, 0, 1, 0, 0, 2, 0, 0, 1, 0, 3, 1, 4, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
0.97 18 [1, 0, 1, 1, 0, 2, 1, 1, 3, 0, 1, 0, 0, 2, 0, 0, 1, 0, 3, 1, 4, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
0.79 3 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.79 3 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.79 3 